In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/wine-information/cleansingWine.csv').drop('Unnamed: 0', axis=1)
df

In [ ]:
for i in range(1,6):
    i = str(i)
    df['sweet'] = df['sweet'].replace(('SWEET'+i),i)
    df['acidity'] = df['acidity'].replace(('ACIDITY'+i),i)
    df['body'] = df['body'].replace(('BODY'+i),i)
    df['tannin'] = df['tannin'].replace(('TANNIN'+i),i)
df

# for Preprocessing

In [ ]:
df = df.fillna(0)

In [ ]:
abv = []
for i in range(len(df)):
    try:
        x,y = df['abv'][i].split('~')
        abv.append((float(x)+float(y))/2)
    except:
        abv.append(df['abv'][i])
        continue
df.abv = abv

In [ ]:
degree = []
for i in range(len(df)):
    try:
        x,y = df['degree'][i].split('~')
        degree.append((float(x)+float(y))/2)
    except:
        degree.append(df['degree'][i])
        continue
df.degree = degree

# Encoding varieties, nation

In [ ]:
s = set()
for i in df.columns[8:20]:
    for j in df[i].unique():
        s.add(j)

In [ ]:
v_arr = list(s)

In [ ]:
v_dic = {i : v_arr.index(i) for i in v_arr}

In [ ]:
for i in df.columns[8:20]:
    arr = []
    for j in df[i]:
        if j == None:
            arr.append(0)
        else:
            arr.append(v_dic[j])
    df[i] = arr

In [ ]:
s = set()
for j in df['nation'].unique():
    s.add(j)
n_arr = list(s)
n_dic = {i : n_arr.index(i) for i in n_arr}

In [ ]:
n_dic

In [ ]:
arr = []
for i in df['nation']:
    if i == None:
        arr.append(0)
    else:
        arr.append(n_dic[i])
df['nation'] = arr

# for Compute similarity

In [ ]:
tmp = df[['nation', 'varieties1','varieties2', 'varieties3', 'varieties4', 'varieties5', 'varieties7', 'varieties7', 
          'varieties8', 'varieties9', 'varieties10', 'varieties11', 'varieties12', 'abv', 'degree', 'sweet', 'acidity', 'body', 'tannin']]
tmp

In [ ]:
tmp['abv'] = tmp['abv'].astype('float64')
tmp['degree'] = tmp['degree'].astype('float64')
tmp['sweet'] = tmp['sweet'].astype('int64')
tmp['acidity'] = tmp['acidity'].astype('int64')
tmp['body'] = tmp['body'].astype('int64')
tmp['tannin'] = tmp['tannin'].astype('int64')

In [ ]:
tmp.info()

# Define Similarity Function

In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

# 코사인 유사도 함수 
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [ ]:
def manhattan_distance(pt1, pt2):
  distance = 0
  for i in range(len(pt1)):
    distance += abs(pt1[i] - pt2[i])
  return distance

# Compare df[0] with All

In [ ]:
df.head(1)

In [ ]:
sim = []
for i in range(1, len(tmp)):
    sim.append(cos_sim(tmp.iloc[0].values, tmp.iloc[i].values))

In [ ]:
man = []
for i in range(1, len(tmp)):
    man.append(manhattan_distance(tmp.iloc[0].values, tmp.iloc[i].values))

In [ ]:
coss = pd.DataFrame({'id' : df['id'][1:].tolist(), 
             'sim' : sim})

In [ ]:
mans = pd.DataFrame({'id' : df['id'][1:].tolist(), 
             'sim' : man})

# 10 High similarity 

In [ ]:
# cos
pd.concat([df[df['id'] != 137197].reset_index().drop('index', axis=1), coss], axis=1).sort_values(by=['sim'], ascending=False).head(10)

In [ ]:
# manhattan
pd.concat([df[df['id'] != 137197].reset_index().drop('index', axis=1), mans], axis=1).sort_values(by=['sim'], ascending=True).head(10)